Wine Reviews Data Set from Kaggle: https://www.kaggle.com/zynicide/wine-reviews/version/4#winemag-data-130k-v2.csv

In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd
import os
import re
import csv

In [2]:
reviews_train = []
test_file='C:/Users/Chaitra C Mouli/Downloads/wine_reviews_test.csv'
train_file='C:/Users/Chaitra C Mouli/Downloads/wine_reviews.csv'


In [3]:
columns=[]
with open(train_file) as f:
    reader = csv.reader(f)
    reader.next()
    for row in reader:
        columns.append(row)
reviews_train=[col[2] for col in columns]


In [4]:
columns=[]
with open(test_file) as f:
    reader = csv.reader(f)
    reader.next()
    for row in reader:
        columns.append(row)
reviews_test=[col[2] for col in columns]

In [5]:
reviews_train[0]

"Moorooduc's estate Pinot Noir is a solid value among the boutique offerings from the Mornington Peninsula. There's a supple cushioning of flesh wrapped around a firm backbone of acid, while the flavors run from black cherry and mint through earthier notes of brown sugar and roasted parsnip. Drink now through 2020."

In [6]:
reviews_test[0]

'This Bordeaux-style blend is comprised of 44% Cabernet Sauvignon, 43% Merlot and 13% Cabernet Franc. The fruit flavors of raspberry and blackberry are light and tangy, the tannins mellow. A note of dark chocolate highlights the finish.'

In [7]:
import re

REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])|(\d+)")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
NO_SPACE = ""
SPACE = " "

def preprocess_reviews(reviews):
    
    reviews = [REPLACE_NO_SPACE.sub(NO_SPACE, line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(SPACE, line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

In [8]:
len(reviews_train_clean)

4957

In [9]:
len(reviews_test_clean)

4957

In [10]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)
cv.fit(reviews_train_clean)
X = cv.transform(reviews_train_clean)
X_test = cv.transform(reviews_test_clean)

In [11]:
X

<4957x6582 sparse matrix of type '<type 'numpy.int64'>'
	with 182382 stored elements in Compressed Sparse Row format>

In [12]:
X_test

<4957x6582 sparse matrix of type '<type 'numpy.int64'>'
	with 151570 stored elements in Compressed Sparse Row format>

In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

target = [1 if i < 2478 else 0 for i in range(4957)]

X_train, X_val, y_train, y_val = train_test_split(X, target)
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    

Accuracy for C=0.01: 0.5943548387096774
Accuracy for C=0.05: 0.5991935483870968
Accuracy for C=0.25: 0.5951612903225807
Accuracy for C=0.5: 0.5943548387096774
Accuracy for C=1: 0.5814516129032258


In [14]:
final_model = LogisticRegression(C=0.05)
final_model.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_model.predict(X_test)))

Final Accuracy: 0.5604196086342546


In [17]:
feature_to_coef = {
    word: coef for word, coef in zip(
        cv.get_feature_names(), final_model.coef_[0]
    )
}
for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:10]:
    print (best_positive)

(u'displays', 0.39715032047221077)
(u'ample', 0.35072039367140206)
(u'baking', 0.3312345215822537)
(u'sparkling', 0.30218687157016144)
(u'drink', 0.272054077000183)
(u'least', 0.2719198427549324)
(u'modest', 0.2650772633718504)
(u'noir', 0.24088158059669362)
(u'barossa', 0.2400955818608406)
(u'own', 0.2399563684027594)


In [18]:
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:10]:
    print (best_negative)

(u'cab', -0.5672479549916192)
(u'offers', -0.3353248965391287)
(u'powerful', -0.3211406846318346)
(u'eucalyptus', -0.3077726117407616)
(u'smoky', -0.2858078108345792)
(u'still', -0.2762619568278298)
(u'tartness', -0.2723747899004203)
(u'everyday', -0.2662177320779585)
(u'darker', -0.2579956615184325)
(u'tobacco', -0.25317007757522897)
